In [31]:
import pandas as pd
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
import psycopg2.extras as extras
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [32]:
recipe_df = pd.read_csv('./recipe_df.csv')
ingr_df = pd.read_csv('./ingr_df.csv')

### Specify the connection parameters

In [33]:
conn_params_dic = {
    "host"      : "localhost",
    "database"  : "recipesdb",
    "user"      : "postgres",
    "password"  : "*"
}

### Support Functions

In [34]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

        
# Define function using psycopg2.extras.execute_values() to insert the dataframe.
def execute(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, sql, tpls)
        conn.commit()
        print("Data inserted using execute_values() successfully...")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

### Perform main task

In [35]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, recipe_df, 'recipes')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


### Perform main task

In [36]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
# Run the execute_many method
execute(conn, ingr_df, 'ingredients')
# Close the connection
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using execute_values() successfully...


### Query the database to check

In [37]:
# Connect to the database
conn = connect(conn_params_dic)
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM recipes"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# list of columns
cols = list(recipe_df.columns)

recipe_df_read = pd.DataFrame(tuples,columns=cols)

display(recipe_df_read.head(3))

# Close the cursor
cursor.close()

# Close the conn
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................


,recipe_id,recipe_name,ingredients,quantity,text
0,0,Курица с кабачками в духовке рецепт на 4 порци...,Куриное филе,500 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
1,0,Курица с кабачками в духовке рецепт на 4 порци...,Кабачки,250 гр.,"Куриное филе нарезать тонкими ломтиками, после..."
2,0,Курица с кабачками в духовке рецепт на 4 порци...,Лук,180 гр.,"Куриное филе нарезать тонкими ломтиками, после..."


### Query the database to check

In [38]:
# Connect to the database
conn = connect(conn_params_dic)
cursor = conn.cursor()

# Execute query
sql = "SELECT * FROM ingredients"
cursor.execute(sql)

# Fetch all the records
tuples = cursor.fetchall()

# list of columns
cols = list(ingr_df.columns)

ingredients_df_read = pd.DataFrame(tuples,columns=cols)

display(ingredients_df_read.head())

# Close the cursor
cursor.close()

# Close the conn
conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................


,ingr_id,ingr_name
0,0,Куриное филе
1,1,Кабачки
2,2,Лук
3,3,Сыр
4,4,Чеснок
